In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lid_driven_cavity import operators, states
import numpy as np

import scipy as sp
np.set_printoptions(linewidth=2000, threshold=5000, precision=6)

In [3]:
# Physics parameters
reynolds_numbers = [100, 400, 1000, 3200, 5000]

# Boundary conditions
u_top = (1, 0)
u_bot = (0, 0)
u_left = (0, 0)
u_right = (0, 0)

# Spatial parameters
Lx = 1.0
Ly = 1.0

In [4]:
# Physics parameter
Re = reynolds_numbers[-1]

# Spatial resolution
N = 3  # number of cells in each direction
h = 1.0 / N

# Temporal resolution
k = 1.0  # FIXME arbitrary for now, likely will need to decrease

# Setup

1. Assume $\phi^n$ and $u^n$ are known at all interior, boundary, and ghost points.

In [5]:
phi_n = states.State(np.zeros((N+2, N+2)), Nx=N+2, Ny=N+2)
u_n = states.State(np.zeros((N+2, N+1)), Nx=N+1, Ny=N+2)
v_n = states.State(np.zeros((N+1, N+2)), Nx=N+2, Ny=N+1)
print(phi_n)
print(u_n)
print(v_n)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


2. Assume $G(\phi^n)$ is known at all interior, boundary, and ghost points.

In [6]:
Gx_phi_n = operators.compute_gradient_x(phi_n)  # on phi grid, no left boundary
Gy_phi_n = operators.compute_gradient_y(phi_n)  # on phi grid, no bottom boundary
print(Gx_phi_n)
print(Gy_phi_n)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


3. Assume $N(u^{n−1})$ is known at all interior points.

In [7]:
Nu_nm1 = states.State(np.zeros((N+2, N-1+2)), Nx=N-1+2, Ny=N+2)
Nv_nm1 = states.State(np.zeros((N-1+2, N+2)), Nx=N+2, Ny=N-1+2)
print(Nu_nm1)
print(Nv_nm1)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


# Prediction step

4. Compute $\hat{u}^n$ at all interior points.

In [8]:
u_hat_n = operators.compute_interpolated_x(u_n)  # on phi grid, no boundary columns
v_hat_n = operators.compute_interpolated_y(v_n)  # on phi grid, no boundary rows
print(u_hat_n)
print(v_hat_n)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


5. Compute $N(u^n)$ at all interior points.

In [9]:
Nu = operators.compute_Nu(u_n, v_hat_n, h)
Nv = operators.compute_Nv(v_n, u_hat_n, h)
print(Nu)
print(Nv)

[[ 0.  0.  0.  0.]
 [ 0. -0. -0.  0.]
 [ 0. -0. -0.  0.]
 [ 0. -0. -0.  0.]
 [ 0.  0.  0.  0.]]
[[ 0.  0.  0.  0.  0.]
 [ 0. -0. -0. -0.  0.]
 [ 0. -0. -0. -0.  0.]
 [ 0.  0.  0.  0.  0.]]


6. Evaluate $g(x, t_{n+1})$ at all required locations on the domain boundary.

7. Solve for $u^*$ at all interior, boundary, and ghost points.

In [10]:
Lu = operators.compute_laplace(u_n)
Lv = operators.compute_laplace(v_n)
print(Lu)
print(Lv)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [11]:
# Assemble RHS
b_interior_u = (k/2) * (3*Nu.get_matrix()-Nu_nm1.get_matrix()) + u_n.get_matrix() + (k/2)*(1/Re)*Lu.get_matrix()
b_interior_u = states.strip_boundaries(b_interior_u)

# pad with zeros on boundaries
b_u = np.zeros((b_interior_u.shape[0]+2, b_interior_u.shape[1]+2))
b_u[1:-1, 1:-1] = b_interior_u

# convert to vector
b_u = states.matrix_to_vector(b_u)

# Assemble operator
# TODO separate assemble...() into a basic (full) laplace operator and a functionality to edit for BCs
laplacian, b = operators.assemble_laplacian_operator_u(b_u, Nx=N+1, Ny=N+2, h=h)

# Assemble linear system
A1 = sp.sparse.eye(laplacian.shape[0], dtype=float, format='csr')
A2 = (k/2)*(1/Re)*laplacian.get_csr()
A = A1 + A2

print(A.toarray())
print(b)

# Solve implicit step
u_star = sp.sparse.linalg.spsolve(A, b)
u_star = states.State(u_star, Nx=4, Ny=5)
print(u_star)

[[1.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 1.0001e+00 0.0000e+00 0.0000e+00 0.0000e+00 1.0000e-04 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 0.0000e+00 1.0001e+00 0.0000e+00 0.0000e+00 0.0000e+00 1.0000e-04 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 0.0000e+00 0.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 1.0001e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e

In [12]:
# Assemble RHS
b_interior_v = (k/2) * (3*Nv.get_matrix()-Nv_nm1.get_matrix()) + v_n.get_matrix() + (k/2)*(1/Re)*Lv.get_matrix()
b_interior_v = states.strip_boundaries(b_interior_v)

# pad with zeros on boundaries
b_v = np.zeros((b_interior_v.shape[0]+2, b_interior_v.shape[1]+2))
b_v[1:-1, 1:-1] = b_interior_v

# convert to vector
b_v = states.matrix_to_vector(b_v)

# Assemble operator
# TODO separate assemble...() into a basic (full) laplace operator and a functionality to edit for BCs
laplacian, b = operators.assemble_laplacian_operator_v(b_v, Nx=N+2, Ny=N+1, h=h)

# Assemble linear system
A1 = sp.sparse.eye(laplacian.shape[0], dtype=float, format='csr')
A2 = (k/2)*(1/Re)*laplacian.get_csr()
A = A1 + A2

print(A.toarray())
print(b)

# Solve implicit step
v_star = sp.sparse.linalg.spsolve(A, b)
v_star = states.State(v_star, Nx=5, Ny=4)
print(v_star)

[[1.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 1.0001e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 0.0000e+00 1.0001e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 0.0000e+00 0.0000e+00 1.0001e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e

# Poisson problem

8. Compute $D(u^*)$ at all interior points.

In [13]:
u_star_matrix = u_star.get_matrix()
v_star_matrix = v_star.get_matrix()
print(u_star_matrix)
print(v_star_matrix)

[[ 0.000000e+00  1.999800e+00  1.999800e+00  0.000000e+00]
 [ 0.000000e+00 -1.804694e-03 -1.804694e-03  0.000000e+00]
 [ 0.000000e+00  1.628623e-06  1.628623e-06  0.000000e+00]
 [ 0.000000e+00 -1.469730e-09 -1.469730e-09  0.000000e+00]
 [ 0.000000e+00  1.469583e-13  1.469583e-13  0.000000e+00]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [14]:
# TODO refactor and verify order of convergence
dudx = u_star_matrix[:, 1:] - u_star_matrix[:, :-1]
dvdy = v_star_matrix[1:, :] - v_star_matrix[:-1, :]
print(dudx)
print(dvdy)

dudx_interior = dudx[1:-1, :]
dvdy_interior = dvdy[:, 1:-1]
print(dudx_interior)
print(dvdy_interior)

Dstar = (dudx_interior + dvdy_interior) / h
print(Dstar)

[[ 1.999800e+00  0.000000e+00 -1.999800e+00]
 [-1.804694e-03  2.168404e-19  1.804694e-03]
 [ 1.628623e-06  2.117582e-22 -1.628623e-06]
 [-1.469730e-09  0.000000e+00  1.469730e-09]
 [ 1.469583e-13  0.000000e+00 -1.469583e-13]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[-1.804694e-03  2.168404e-19  1.804694e-03]
 [ 1.628623e-06  2.117582e-22 -1.628623e-06]
 [-1.469730e-09  0.000000e+00  1.469730e-09]]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[-5.414083e-03  6.505213e-19  5.414083e-03]
 [ 4.885870e-06  6.352747e-22 -4.885870e-06]
 [-4.409189e-09  0.000000e+00  4.409189e-09]]


9. Solve for $\phi^{n+1}$ at all interior and ghost points.

In [24]:
b_interior = Dstar / k

# pad with zeros
b = np.zeros((b_interior.shape[0]+2, b_interior.shape[1]+2))
b[1:-1, 1:-1] = b_interior
print(b)

# convert to vector
b = states.matrix_to_vector(b)

# TODO add ones on main diagonal for state_ij = 0 entries!!!
Lphi, b = operators.assemble_laplacian_operator_phi(b, u_star_matrix, v_star_matrix, Nx=N+2, Ny=N+2, h=h, k=k)
A = Lphi.get_csr()
print(A.toarray())

# phi_np1 = sp.sparse.linalg.spsolve(A, b)
phi_np1 = sp.sparse.linalg.gmres(A, b)
print(phi_np1)
phi_np1 = states.State(phi_np1[0], Nx=N+2, Ny=N+2)
# phi_np1 = sp.sparse.linalg.cg(A, b)
# print(phi_np1)

[[ 0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00]
 [ 0.000000e+00 -5.414083e-03  6.505213e-19  5.414083e-03  0.000000e+00]
 [ 0.000000e+00  4.885870e-06  6.352747e-22 -4.885870e-06  0.000000e+00]
 [ 0.000000e+00 -4.409189e-09  0.000000e+00  4.409189e-09  0.000000e+00]
 [ 0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00]]
[[  1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   1.   0.   0.   0.   0.  -1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   1.   0.   0.   0.   0.  -1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   1.   0.   0.   0.   0.  -1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. 

# Correction step

10. Compute $G(\phi^{n+1})$ at all interior, boundary, and ghost points.

In [26]:
Gx_phi_np1 = operators.compute_gradient_x(phi_np1)
Gy_phi_np1 = operators.compute_gradient_y(phi_np1)

11. Correct $u^*$ to obtain $u^{n+1}$ at all interior, boundary, and ghost points.

In [38]:
u_star_matrix = u_star.get_matrix()
Gx_phi_np1_matrix = Gx_phi_np1.get_matrix()
Gx_phi_np1_matrix = Gx_phi_np1_matrix[:, 1:]
print(u_star_matrix)
print(Gx_phi_np1_matrix)

v_star_matrix = v_star.get_matrix()
Gy_phi_np1_matrix = Gy_phi_np1.get_matrix()
Gy_phi_np1_matrix = Gy_phi_np1_matrix[:-1, :]
print(v_star_matrix)
print(Gy_phi_np1_matrix)

[[ 0.000000e+00  1.999800e+00  1.999800e+00  0.000000e+00]
 [ 0.000000e+00 -1.804694e-03 -1.804694e-03  0.000000e+00]
 [ 0.000000e+00  1.628623e-06  1.628623e-06  0.000000e+00]
 [ 0.000000e+00 -1.469730e-09 -1.469730e-09  0.000000e+00]
 [ 0.000000e+00  1.469583e-13  1.469583e-13  0.000000e+00]]
[[ 3.006749e-03 -1.503398e-03 -1.503339e-03 -1.131597e-08]
 [-1.070200e-08 -1.503376e-03 -1.503362e-03  7.383131e-10]
 [ 3.588957e-08 -6.004734e-04 -6.004734e-04  1.037016e-09]
 [-1.304957e-08 -3.002519e-04 -3.002311e-04  5.430077e-09]
 [ 1.803623e-03 -3.002881e-04 -3.002025e-04 -1.203132e-03]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[-3.006749e-03  1.070200e-08 -1.169794e-08  1.073865e-08 -1.315625e-09]
 [ 9.029513e-04  9.029047e-04  2.376824e-09 -9.028859e-04 -9.028862e-04]
 [ 3.001749e-04  3.002238e-04  2.352492e-09 -3.002400e-04 -3.002444e-04]
 [ 1.803623e-03 -1.304957e-08  2.315375e-08 -5.430077e-09  1.203132e-03]]


In [40]:
u_np1_matrix = u_star_matrix - k * Gx_phi_np1_matrix
v_np1_matrix = v_star_matrix - k * Gy_phi_np1_matrix
print(u_np1_matrix)
print(v_np1_matrix)

[[-3.006749e-03  2.001304e+00  2.001304e+00  1.131597e-08]
 [ 1.070200e-08 -3.013185e-04 -3.013326e-04 -7.383131e-10]
 [-3.588957e-08  6.021020e-04  6.021020e-04 -1.037016e-09]
 [ 1.304957e-08  3.002505e-04  3.002296e-04 -5.430077e-09]
 [-1.803623e-03  3.002881e-04  3.002025e-04  1.203132e-03]]
[[ 3.006749e-03 -1.070200e-08  1.169794e-08 -1.073865e-08  1.315625e-09]
 [-9.029513e-04 -9.029047e-04 -2.376824e-09  9.028859e-04  9.028862e-04]
 [-3.001749e-04 -3.002238e-04 -2.352492e-09  3.002400e-04  3.002444e-04]
 [-1.803623e-03  1.304957e-08 -2.315375e-08  5.430077e-09 -1.203132e-03]]


# Bookkeeping

12. Update counter from $n$ to $n + 1$ and time from $t_n$ to $t_{n+1}$.

In [18]:
n = n + 1
tn = tn + k

In [41]:
phi_n = states.State(phi_np1, Nx=phi_n.Nx, Ny=phi_n.Ny)
u_n = states.State(u_np1_matrix, Nx=u_n.Nx, Ny=u_n.Ny)
v_n = states.State(v_np1_matrix, Nx=v_n.Nx, Ny=v_n.Ny)

[[0.000000e+00 7.516872e-04 3.758376e-04 2.828991e-09 0.000000e+00]
 [7.516872e-04 7.516845e-04 3.758406e-04 1.443279e-10 3.289062e-10]
 [5.259494e-04 5.259583e-04 3.758400e-04 2.257216e-04 2.257219e-04]
 [4.509056e-04 4.509024e-04 3.758394e-04 3.007816e-04 3.007830e-04]
 [0.000000e+00 4.509056e-04 3.758336e-04 3.007830e-04 0.000000e+00]]


In [49]:
Gx_phi_n = Gx_phi_np1
Gy_phi_n = Gy_phi_np1

In [50]:
Nu_nm1 = Nu
Nv_nm1 = Nv

13. Repeat from Step 1.